### Import the relevant date

In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

### Data

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [3]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

### Shuffeling the data sets from preprocessing

In [4]:
BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))


## Model

### Outline the model

In [5]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

### Choose the optimazer and the loss function

In [6]:
# customizer_optimizer = tf.keras.optimizers.Adam(learning_rate=0.02)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [7]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 8s - loss: 0.2758 - accuracy: 0.9209 - val_loss: 0.1378 - val_accuracy: 0.9602 - 8s/epoch - 15ms/step
Epoch 2/5
540/540 - 5s - loss: 0.1029 - accuracy: 0.9683 - val_loss: 0.0919 - val_accuracy: 0.9712 - 5s/epoch - 10ms/step
Epoch 3/5
540/540 - 5s - loss: 0.0686 - accuracy: 0.9789 - val_loss: 0.0682 - val_accuracy: 0.9798 - 5s/epoch - 10ms/step
Epoch 4/5
540/540 - 5s - loss: 0.0512 - accuracy: 0.9841 - val_loss: 0.0553 - val_accuracy: 0.9825 - 5s/epoch - 10ms/step
Epoch 5/5
540/540 - 5s - loss: 0.0372 - accuracy: 0.9883 - val_loss: 0.0457 - val_accuracy: 0.9847 - 5s/epoch - 10ms/step


### Test the model

In [8]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 864ms/step - loss: 0.0787 - accuracy: 0.9765


In [9]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.08. Test accuracy: 97.65%
